Copyright 2021-2023 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of MindSPONGE:
MindSpore Simulation Package tOwards Next Generation molecular modelling.

MindSPONGE is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

MindSPONGE advanced tutorial 03: Energy wrapper and integrated tempering sampling (ITS).

In [1]:
from mindspore import context
from sponge import Sponge
from sponge import ForceField
from sponge import set_global_units
from sponge import Protein
from sponge import WithEnergyCell
from sponge import UpdaterMD
from sponge.optimizer import SteepestDescent
from sponge.callback import WriteH5MD, RunInfo
from sponge.sampling import ITS
from sponge.function import VelocityGenerator
from sponge.colvar import Torsion

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")
set_global_units('nm', 'kj/mol')

In [3]:
system = Protein(pdb='alad.pdb')
potential = ForceField(system, 'AMBER.FF14SB')

In [4]:
phi = Torsion([4, 6, 8, 14])
psi = Torsion([6, 8, 14, 16])

In [5]:
min_opt = SteepestDescent(system.trainable_params(), 1e-7)
mini = Sponge(system, potential, min_opt, metrics={'phi': phi, 'psi': psi})

[MindSPONGE] Using fully connected neighbour list (not updated).


In [6]:
run_info = RunInfo(10)
mini.run(100, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-05-19 10:23:42
[MindSPONGE] Step: 0, E_pot: -55.75643, phi: -3.140886, psi: -3.1410766
[MindSPONGE] Step: 10, E_pot: -58.40555, phi: -3.1340268, psi: -3.1368413
[MindSPONGE] Step: 20, E_pot: -60.168476, phi: -3.1277595, psi: -3.1337605
[MindSPONGE] Step: 30, E_pot: -61.549706, phi: -3.1221528, psi: -3.131402
[MindSPONGE] Step: 40, E_pot: -62.696198, phi: -3.1171532, psi: -3.1295478
[MindSPONGE] Step: 50, E_pot: -63.674088, phi: -3.112676, psi: -3.1280725
[MindSPONGE] Step: 60, E_pot: -64.52254, phi: -3.1086385, psi: -3.1268952
[MindSPONGE] Step: 70, E_pot: -65.26811, phi: -3.1049678, psi: -3.1259575
[MindSPONGE] Step: 80, E_pot: -65.93031, phi: -3.1016028, psi: -3.1252148
[MindSPONGE] Step: 90, E_pot: -66.52346, phi: -3.0984938, psi: -3.1246321
[MindSPONGE] Finished simulation at 2023-05-19 10:23:46
[MindSPONGE] Simulation time: 3.78 seconds.
--------------------------------------------------------------------------------


In [7]:
its = ITS(
    sim_temp=300,
    temp_min=270,
    temp_max=470,
    temp_bin=100,
    update_pace=100,
    unlinear_temp=True,
)

In [8]:
sim = WithEnergyCell(system, potential, wrapper=its)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [9]:
vgen = VelocityGenerator(300)
velocity = vgen(system.shape, system.atom_mass)
opt = UpdaterMD(
    system=system,
    time_step=1e-3,
    velocity=velocity,
    integrator='velocity_verlet',
    temperature=300,
    thermostat='langevin',
)

In [10]:
md = Sponge(sim, optimizer=opt, metrics={'phi': phi, 'psi': psi})

In [11]:
cb_h5md = WriteH5MD(system, 'tutorial_a03.h5md', save_freq=10)
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-05-19 10:23:52
[MindSPONGE] Step: 0, E_pot: -79.28045, E_kin: 96.96472, E_tot: 17.684273, Temperature: 370.2278, phi: -3.0946958, psi: -3.1193402
[MindSPONGE] Step: 10, E_pot: -99.59543, E_kin: 35.44615, E_tot: -64.14928, Temperature: 135.33943, phi: -3.079391, psi: -3.0574105
[MindSPONGE] Step: 20, E_pot: -111.30186, E_kin: 49.054054, E_tot: -62.247803, Temperature: 187.29672, phi: -3.0604281, psi: -2.9846
[MindSPONGE] Step: 30, E_pot: -108.976944, E_kin: 45.604313, E_tot: -63.37263, Temperature: 174.12502, phi: -3.1081102, psi: -2.9990194
[MindSPONGE] Step: 40, E_pot: -119.62045, E_kin: 65.65898, E_tot: -53.96147, Temperature: 250.69716, phi: 3.1262376, psi: -3.109618
[MindSPONGE] Step: 50, E_pot: -102.952934, E_kin: 62.735622, E_tot: -40.21731, Temperature: 239.53528, phi: -3.1144304, psi: 3.1358683
[MindSPONGE] Step: 60, E_pot: -109.93755, E_kin: 72.00345, E_tot: -37.934105, Temperature: 274.92142, phi: -2.9780388, psi: -3.0246732
[MindSPONGE